In [305]:
import requests
import json
from requests.auth import HTTPBasicAuth
import numpy as np

In [428]:
def feedback_to_tensor(feedback):
    ndarray = np.array(feedback["response"]["response"]["ndarray"])
    feedback["response"]["response"]["tensor"] = {"shape":ndarray.shape,"values":list(ndarray.ravel())}
    del feedback["response"]["response"]["ndarray"]

class ClientException(Exception):
    pass
    
class OAuthClient(object):
    def __init__(self,endpoint,client_key,client_secret):
        self.endpoint = endpoint
        self.token = requests.post(
            "http://{}:{}@{}/oauth/token".format(client_key,client_secret,endpoint),
            headers={"type":"application/json"},
            data={"grant_type": "client_credentials"}).json()['access_token']
    
    def _request(self,method,url,data=None):
        if data is None:
            data = {}
        request = {
            'GET':requests.get,
            'POST':requests.post,
            'DELETE':requests.delete,
            'PUT':requests.put
        }
        response = request[method](
            "http://"+self.endpoint+url, 
            headers={
                "Content-Type":"application/json",
                "Authorization":"Bearer {}".format(self.token)},
            data=json.dumps(data))
        if response.status_code/100!=2:
            raise ClientException(response.text)
        try:
            return response.json()
        except ValueError:
            return response.text
    
    def _get(self,url,data=None):
        return self._request("GET",url,data)
    
    def _post(self,url,data=None):
        return self._request("POST",url,data)
    
    def _delete(self,url,data=None):
        return self._request("DELETE",url,data)
    
    def _put(self,url,data=None):
        return self._request("PUT",url,data)

class ClusterManagerClient(OAuthClient):
    def ping(self):
        return self._get("/ping")
    
    def authping(self):
        return self._get("/api/v1/authping")
    
    def create_deployment(self,deployment):
        response = self._post("/api/v1/deployments",data=deployment)
        return response
    
    def delete_deployment(self,deployment_id):
        response= self._delete("/api/v1/deployments/{}".format(deployment_id))
        return response

class APIFrontEndClient(OAuthClient):
    def ping(self):
        return self._get("/ping")
    
    def predictions(self,request):
        response = self._post("/api/v0.1/predictions",data=request)
        return response
    
    def feedback(self,feedback):
        response = self._post("/api/v0.1/feedback",data=feedback)
        return response

In [475]:
with open("./CLUSTER_MANAGER_ENDPOINT",'r') as f:
    cm_endpoint = f.read()[:-1]

with open("../cluster-manager/cluster-manager-client-secret.txt",'r') as f:
    cm_client_secret = f.readline()[:-1]

with open("./API_ENDPOINT",'r') as f:
    api_endpoint  = f.read()[:-1]

with open(filepath,'r') as f:
    deployment = json.loads(f.read())
        
api_client_key = deployment["oauth_key"]
api_client_secret = deployment["oauth_secret"]

In [525]:
cmc = ClusterManagerClient(cm_endpoint,"client",cm_client_secret)

In [526]:
cmc.ping()

u'pong'

In [527]:
cmc.authping()

u'authpong'

In [503]:
cmc.delete_deployment(1)

{u'cmstatus': {u'code': 200,
  u'info': u'',
  u'reason': u'',
  u'status': u'SUCCESS'}}

In [528]:
created_deployment = cmc.create_deployment(deployment)

In [529]:
apic = APIFrontEndClient(api_endpoint,api_client_key,api_client_secret)

In [530]:
apic.ping()

u'pong'

In [453]:
request = {
    'request':{
        'features':['a','b'],
        'ndarray':[[6,3]]
    },
    'meta':{
        'puid':'0',
        'tags':{}
    }
}

In [454]:
response = apic.predictions(request)

In [455]:
response

{u'meta': {u'puid': u'0', u'routing': {u'0': 2}},
 u'response': {u'features': [u'proba'], u'ndarray': [[0.8296760813561542]]}}

In [456]:
feedback = {
    "request":request,
    "response":response,
    "reward":1
}

feedback_to_tensor(feedback)

In [458]:
apic.feedback(feedback)

u''

In [459]:
def compute_reward(response):
    routing = response['meta']['routing']['0']
    probabilities = {
        0:0.75,
        1:0.89,
        2:0.53
    }
    return int(np.random.rand()<probabilities[routing])

In [461]:
def test(n_queries = 1000):
    errors = []
    rewards = []
    routings = []
    for i in range(n_queries):
        if i%100==0:
            print i
        try:
            response = apic.predictions(request)
            reward = compute_reward(response)
            feedback = {
                "request":request,
                "response":response,
                "reward":reward
            }
            feedback_to_tensor(feedback)
        
            apic.feedback(feedback)
            
            rewards.append(reward)
            routings.append(response['meta']['routing']['0'])
            errors.append(False)
        except ClientException:
            rewards.append(np.nan)
            routings.append(np.nan)
            errors.append(True)
    print "There was {} errors".format(np.sum(errors))
    return np.array(rewards),np.array(routings),np.array(errors)

In [537]:
rewards, routings, errors = test(n_queries=1000)

0
100
200
300
400
500
600
700
800
900
There was 0 errors


In [538]:
routings

array([1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 2, 1, 1,
       2, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 2, 1, 0, 1, 2, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1,

In [539]:
(routings==0).sum()

73

In [540]:
(routings==1).sum()

921

In [541]:
(routings==2).sum()

6

In [542]:
import redis
import cPickle as pickle

In [505]:
client = redis.StrictRedis(host="localhost")

In [543]:
client.keys()

['auth:a39059ce-d297-4fea-8f4e-b48477980baa',
 'client_id_to_access:key',
 'access:97ad2793-73ec-407d-9e12-8cccc00a3542',
 'auth_to_access:a7b7ee6b56e8fcf90c36545d897ea9b9',
 'access:a39059ce-d297-4fea-8f4e-b48477980baa',
 'ts_test0_0',
 'auth_to_access:ee16db94e195840785e8cd7b3456eff4',
 'auth:97ad2793-73ec-407d-9e12-8cccc00a3542',
 'client_id_to_access:client']

In [544]:
pickle.loads(client.get("ts_test0_0"))

{'models_beta_params': [[55, 14], [688, 80], [3, 5]]}